# 07 — Daily report (end-to-end, using Universe Provider)

This notebook runs the full daily pipeline and exports a CSV.

New: tickers are loaded from the Universe Provider:
- `load_universe_from_package("mega")`
- optionally capped via `max_tickers` (like CLI `--top`)

Pipeline:
1) fetch OHLCV
2) build eligible universe
3) rank top N
4) compute entry signals
5) compute position sizing
6) generate final report

In [ ]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## 1) Load universe tickers

In [ ]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=50))
tickers[:15], len(tickers)

## 2) Fetch market data

In [ ]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))
ohlcv.tail()

## 3) Build daily report (end-to-end)

In [ ]:
from swing_screener.reporting.report import build_daily_report, export_report_csv, today_actions, ReportConfig
from swing_screener.risk.position_sizing import RiskConfig

cfg = ReportConfig(
    risk=RiskConfig(account_size=500, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60),
    only_active_signals=False,
)

report = build_daily_report(ohlcv, cfg)
report.head(20)

## 4) Print a short action summary

In [ ]:
print(today_actions(report, max_rows=10))

## 5) Export CSV

In [ ]:
export_path = export_report_csv(report, "out/daily_report.csv")
export_path